### Prepare dependencies
I had issues with selenium and my chromedriver, I had to install the exact dependencies below to make it work.
First add the dependency selenium by executing folowing command
``` bash
uv pip install selenium==4.11.2
uv pip install urllib3==1.26.16
```
***Do not forget to restart the Jupyter kernel to make the package available.***

### Prefered Web Browser
This script will use Safari on MacOSX, please install the Safari driver when required [Safari driver](https://webkit.org/blog/6900/webdriver-support-in-safari-10).
It will assume that Edge is used on Windows systems, install the Chrome driver from: [ChromeDriver](https://googlechromelabs.github.io/chrome-for-testing/#stable) 
Feel free to add other browser support when required.

I am on Windows and I extracted the ChromeDriver and put it in my %USERPROFILE%\AppData\Local\Microsoft\WindowsApps folder to ensure that it is available via the PATH. Any folder available in the PATH environment setting will work.
Start cmd and execute set to see all folders in the PATH environment setting.


In [ ]:
# imports

import os
from dotenv import load_dotenv
from scraper import fetch_website_contents
from IPython.display import Markdown, display
from openai import OpenAI
from selenium import webdriver

In [ ]:
def verify_openai_api_key():
    """Verify that the OpenAI API key is set in the environment variables."""
    load_dotenv(override=True)
    api_key = os.getenv('OPENAI_API_KEY')

    if not api_key:
        raise ValueError("OPENAI_API_KEY is not set in environment variables.")
    
    # Dry run with a simple request to verify the key.
    try:
        client = OpenAI(api_key=api_key)
        client.models.list()
    except:
        raise ValueError("Invalid OPENAI_API_KEY.")
    
    return api_key

def get_webdriver():
    """Initialize and return a Selenium WebDriver based on the operating system."""
        
    # Verify the os, use Safari for MacOS, Chrome for others.
    if os.name == 'posix':        
        driver = webdriver.Safari()
    else:
        driver = webdriver.Chrome()
    return driver

def fetch_website_contents_selenium(url: str) -> str:
    """Fetch website contents using Selenium WebDriver."""
    driver = get_webdriver()
    driver.get(url)
    content = driver.page_source
    driver.quit()
    return content

def messages_for(website: str) -> list[dict]:
    return [
        {
            "role": "system",
            "content": """You are a helpful assistant that summarizes website content.
            You can perfectly understand HTML structure and extract meaningful information from it."""
        },
        {
            "role": "user",
            "content": f"Summarize the following website content:\n\n{website}"
        }
    ]

def summarize_website(url: str, api_key: str) -> str:
    content = fetch_website_contents_selenium(url)
    openai_client = OpenAI(api_key=api_key)
    response = openai_client.chat.completions.create(
        model = "gpt-4.1-mini",
        messages = messages_for(content)
    )
    return response.choices[0].message.content



In [ ]:
try:
    api_key = verify_openai_api_key()
    print(summarize_website("https://www.forbes.com/", api_key))
except Exception as e:
    print(f"Error: {e}")